In [25]:
import pathlib
import os
import subprocess
import time
from tqdm import tqdm
import re

# Configuración de paths
current_script = pathlib.Path().resolve()
script_dir = current_script.parent

path_generador = script_dir / "generate_problem.py"
path_dominio = script_dir / "3_2/dron_domain.pddl"
path_problema = script_dir / "3_2/Problemas"
path_planificador = script_dir.parent / "planificadores/optic-clp"
path_planes = script_dir / "3_2/planes_generados"
path_problema.mkdir(exist_ok=True)
path_planes.mkdir(exist_ok=True)
max_time = 60
size = 2
max_size = 0

concurrency_pattern = re.compile(r"\(\s*\(.+?\)\s+\(.+?\)\s*\)")

pbar = tqdm(desc="Testing problem sizes", unit="size")
while True:
    # Generar el archivo de problema
    gen_cmd = [
        "python3", str(path_generador),
        "-d", "2", "-r", "2",
        "-l", str(size), "-p", str(size),
        "-c", str(size), "-g", str(size),
        "-o", str(path_problema)
    ]
    try:
        gen_result = subprocess.run(gen_cmd, check=True, capture_output=True, text=True)
        print("Generator STDOUT:\n", gen_result.stdout)
        print("Generator STDERR:\n", gen_result.stderr)
    except subprocess.CalledProcessError as e:
        print("Generator Error STDOUT:\n", e.stdout)
        print("Generator Error STDERR:\n", e.stderr)
        break

    # Extraer el nombre del archivo generado
    problem_file = None
    for line in gen_result.stdout.splitlines():
        if "writing to" in line:
            problem_file = pathlib.Path(line.split("writing to")[-1].strip())
            break
    if problem_file is None:
        problem_file = path_problema / f"drone_problem_d2_r2_l{size}_p{size}_c{size}_g{size}_ct2.pddl"

    # Ejecutar el planificador
    optic_cmd = [
        str(path_planificador), '-E', '-N',
        str(path_dominio),
        str(problem_file)
    ]
    start_time = time.time()
    try:
        result = subprocess.run(
            optic_cmd,
            timeout=max_time,
            capture_output=True,
            text=True
        )
        print("Planner STDOUT:\n", result.stdout)
        print("Planner STDERR:\n", result.stderr)
        execution_time = time.time() - start_time
        if result.returncode == 0:
            # Detectar acciones concurrentes
            concurrent_actions = any(
                concurrency_pattern.search(line)
                for line in result.stdout.split('\n')
            )
            # Guardar el plan en la carpeta planes_generados
            plan_filename = path_planes / f"plan_size_{size}.txt"
            with open(plan_filename, "w") as f:
                f.write(result.stdout)
            pbar.write(
                f"Size {size}: Solved in {execution_time:.2f}s | "
                f"Concurrency: {concurrent_actions}"
            )
            max_size = size
            size += 1
        else:
            pbar.write(f"Planner failed at size {size} with return code {result.returncode}")
            break

    except subprocess.TimeoutExpired:
        pbar.write(f"Timeout at size {size}")
        break
    except Exception as e:
        print("Unexpected error:", e)
        break

    pbar.update(1)

pbar.close()
print(f"\nMaximum solvable size: {max_size}")


Testing problem sizes: 1size [00:00,  6.79size/s]

Generator STDOUT:
 Drones		 2
Carriers	 2
Locations	 2
Persons		 2
Crates		 2
Goals		 2
Problem name: drone_problem_d2_r2_l2_p2_c2_g2_ct2
debug: writing to /home/javikugan/Documents/Universidad/Tercero/PlaniExt/Plani/Ejercicio3/3_2/Problemas/drone_problem_d2_r2_l2_p2_c2_g2_ct2.pddl
debug: person needs: [[True, True], [False, False]]
Generated problem with action‐costs: /home/javikugan/Documents/Universidad/Tercero/PlaniExt/Plani/Ejercicio3/3_2/Problemas/drone_problem_d2_r2_l2_p2_c2_g2_ct2.pddl

Generator STDERR:
 
Planner STDOUT:
 Number of literals: 54
Constructing lookup tables: [10%] [20%] [30%] [40%] [50%] [60%] [70%] [80%] [90%] [100%]
Post filtering unreachable actions:  [10%] [20%] [30%] [40%] [50%] [60%] [70%] [80%] [90%] [100%]
No analytic limits found, not considering limit effects of goal-only operators
30% of the ground temporal actions in this problem are compression-safe
Initial heuristic = 7.000, admissible cost estimate 0.000

Resorting to best-first search
Running WA* 

Testing problem sizes: 2size [00:00,  2.72size/s]

Planner STDOUT:
 Number of literals: 72
Constructing lookup tables: [10%] [20%] [30%] [40%] [50%] [60%] [70%] [80%] [90%] [100%]
Post filtering unreachable actions:  [10%] [20%] [30%] [40%] [50%] [60%] [70%] [80%] [90%] [100%]
No analytic limits found, not considering limit effects of goal-only operators
26% of the ground temporal actions in this problem are compression-safe
Initial heuristic = 10.000, admissible cost estimate 0.000

Resorting to best-first search
Running WA* with W = 5.000, not restarting with goal states
b (9.000 | 170.002)b (8.000 | 170.002)b (7.000 | 338.003)b (5.000 | 338.003)b (4.000 | 338.003)b (3.000 | 338.003)b (2.000 | 404.008)b (1.000 | 404.008)(G);;;; Solution Found
; States evaluated: 562
; Cost: 484.009
; Time 0.48
0.000: (cargar-dron dron1 crate2 deposito)  [1.000]
0.000: (mover-dron dron2 deposito loc3)  [80.000]
1.001: (mover-dron dron1 deposito loc2)  [168.000]
80.001: (mover-dron dron2 loc3 deposito)  [80.000]
160.002: (cargar-dron dron2 crate3 depos

Testing problem sizes: 3size [00:03,  1.51s/size]

...

Planner STDOUT:
 Number of literals: 138
Constructing lookup tables: [10%] [20%] [30%] [40%] [50%] [60%] [70%] [80%] [90%] [100%]
Post filtering unreachable actions:  [10%] [20%] [30%] [40%] [50%] [60%] [70%] [80%] [90%] [100%]
No analytic limits found, not considering limit effects of goal-only operators
22% of the ground temporal actions in this problem are compression-safe
Initial heuristic = 18.000, admissible cost estimate 0.000

Resorting to best-first search
Running WA* with W = 5.000, not restarting with goal states
b (17.000 | 156.002)b (16.000 | 156.002)b (15.000 | 310.003)b (14.000 | 310.003)b (13.000 | 311.004)b (12.000 | 383.005)b (11.000 | 383.006)b (10.000 | 454.006)b (9.000 | 454.006)b (8.000 | 454.006)b (7.000 | 465.007)b (6.000 | 596.008)b (5.000 | 609.011)b (4.000 | 713.010)b (3.000 | 713.010)b (2.000 | 714.011)b (1.000 | 714.012)(G);;;; Solution Found
; States evaluated: 5495
; Cost: 830.013
; Time 13.87
0.000: (cargar-dron dron1 crate6 deposito)  [1.000]
0.000: (mo

Testing problem sizes: 6size [01:09, 20.46s/size]

Planner STDOUT:
 Number of literals: 164
Constructing lookup tables: [10%] [20%] [30%] [40%] [50%] [60%] [70%] [80%] [90%] [100%]
Post filtering unreachable actions:  [10%] [20%] [30%] [40%] [50%] [60%] [70%] [80%] [90%] [100%]
No analytic limits found, not considering limit effects of goal-only operators
22% of the ground temporal actions in this problem are compression-safe
Initial heuristic = 21.000, admissible cost estimate 0.000

Resorting to best-first search
Running WA* with W = 5.000, not restarting with goal states
b (20.000 | 122.002)b (20.000 | 122.001)b (19.000 | 122.002)b (18.000 | 242.003)b (16.000 | 242.003)b (15.000 | 242.003)b (14.000 | 242.003)b (13.000 | 250.008)b (12.000 | 357.007)b (11.000 | 1000281.006)b (11.000 | 414.007)b (11.000 | 390.007)b (10.000 | 575.008)b (10.000 | 535.008)b (9.000 | 575.008)b (8.000 | 575.008)b (7.000 | 649.012)b (6.000 | 794.012)b (5.000 | 794.012)b (4.000 | 794.012)b (3.000 | 794.012)b (2.000 | 794.012)b (1.000 | 794.012)(G);;;; Solutio

Testing problem sizes: 6size [02:09, 21.54s/size]

Timeout at size 8

Maximum solvable size: 7
